In [1]:
from PIL import Image
import numpy as np

In [2]:
def rgb2gray(rgb):
    r, g, b = rgb[:,:,0], rgb[:,:,1], rgb[:,:,2]
    gray = 0.2989 * r + 0.5870 * g + 0.1140 * b

    return gray.astype(np.uint8)

In [3]:
ar = np.array(Image.open(r"pi_image.png"))

In [4]:
ar.shape

(50, 50)

In [5]:
ar[0]

array([30, 10, 40, 10, 50, 90, 20, 60, 50, 30, 50, 80, 90, 70, 90, 30, 20,
       30, 80, 40, 60, 20, 60, 40, 30, 30, 80, 30, 20, 70, 90, 50,  0, 20,
       80, 80, 40, 10, 90, 70, 10, 60, 90, 30, 90, 90, 30, 70, 50, 10],
      dtype=uint8)

In [6]:
actual_pi = (np.loadtxt('pi.tsv', delimiter=' ', dtype= str).astype(int)*10).reshape((50, 50))


In [7]:
comparison = actual_pi != ar

In [8]:
np.sum(comparison)

4

In [14]:
from math import pi

filter = []

for x in range(50):
    for y in range(50):
        if (comparison)[x][y]:
            print(actual_pi[x][y])
            filter.append(actual_pi[x][y])


filter = np.array(sorted(filter))
filter = filter.reshape(2,2) * pi
filter = filter.astype(np.uint8)

0
80
30
90


In [15]:
filter

array([[  0,  94],
       [251,  26]], dtype=uint8)

In [13]:
Image.fromarray(filter).save(f'filter.png')

In [11]:
im_2 = Image.open(r"artwork_picasso.png")
ar_2 = np.array(im_2)
ar_2.shape

(100, 100)

In [13]:
potentials = np.zeros((3, 100, 100))

for i in range(3):
    for x in range (50):
        for y in range(50):
            potential_img_00 = [filter[0][0] ^ ar_2[2*x][2*y], filter[0][0] | ar_2[2*x][2*y], filter[0][0] & ar_2[2*x][2*y]]
            potential_img_01 = [filter[0][1] ^ ar_2[2*x][2*y + 1], filter[0][1] | ar_2[2*x][2*y + 1],  filter[0][1] & ar_2[2*x][2*y + 1]]
            potential_img_10 = [filter[1][0] ^ ar_2[2*x + 1][2*y],  filter[1][0] | ar_2[2*x + 1][2*y], filter[1][0] & ar_2[2*x + 1][2*y]]
            potential_img_11 = [filter[1][1] ^ ar_2[2*x + 1][2*y + 1], filter[1][1] | ar_2[2*x + 1][2*y + 1], filter[1][1] & ar_2[2*x + 1][2*y + 1]]
            
            potentials[i][2*x][2*y] = potential_img_00[i]
            potentials[i][2*x+ 1][2*y] = potential_img_10[i]
            potentials[i][2*x + 1][2*y + 1] = potential_img_11[i]
            potentials[i][2*x][2*y + 1] = potential_img_01[i]


    data = Image.fromarray(potentials[i].astype(np.uint8))
    # data.convert('L')
    data.save(f'Potentials/potential_{i}.png')

In [15]:
col_ar = rgb2gray(np.array(Image.open('collage.png'))).astype(np.uint8)

Image.fromarray(col_ar).save('greyscale.png')

In [16]:
H = 0

while H < col_ar.shape[0]:
    W = 0
    while W < col_ar.shape[1]:
        window = col_ar[W: W+100, H: H+100]
        if np.sum(window):
            Image.fromarray(
                col_ar[W: W+100, H: H+100]
                ).save(
                    f'collage/col_{int(H/100)}{int(W/100)}.png'
                    )
        W += 100
    H += 100

In [51]:
col_ar

array([[192, 192, 192, ...,   0,   0,   0],
       [192, 192, 192, ...,   0,   0,   0],
       [192, 192, 192, ...,   0,   0,   0],
       ...,
       [  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   0,   0]], dtype=uint8)

In [104]:
template = np.array(Image.open('Potentials/potential_2.png'))

In [105]:
template

array([[  0,  26,   0, ...,  28,   0,  28],
       [154,   0, 154, ...,   2, 152,   2],
       [  0,  26,   0, ...,  28,   0,  68],
       ...,
       [251,   0, 250, ...,  24, 233,  18],
       [  0,  94,   0, ...,  76,   0,  76],
       [250,   0, 250, ...,  24, 226,  24]], dtype=uint8)

In [54]:
import os

In [134]:
def score(img1: np.ndarray, img2: np.ndarray):
    return np.sum(np.sqrt(abs((img1)**2 - (img2)**2))/1000)

In [155]:
from collections import defaultdict

full_w,full_h = col_ar.shape
sub_w,sub_h = template.shape

print(full_w,full_h)
print(sub_w,sub_h)

score_dict = defaultdict(int)
for sub_image_name in os.listdir("Potentials/"):
    if sub_image_name.endswith(".png"):
      # sub_image_name = 'potential_2.png'
      sub_image = np.array(Image.open(f"Potentials/{sub_image_name}"))
      for window_name in os.listdir("Collage/"):
         if window_name.endswith(".png"):
            window = np.array(Image.open(f"Collage/{window_name}"))
            score_dict[window_name, sub_image_name] += score(window, sub_image)

800 800
100 100


In [157]:
sorted(score_dict.items(), key = lambda x: x[1])[0]

(('col_11.png', 'potential_2.png'), 100.8006820678711)

In [168]:
import re

x= re.match(pattern='col_([0-9])([0-9]).png', string=sorted(score_dict.items(), key = lambda x: x[1])[0][0][0])
password = int((int(x[1])*100 + int(x[2])*100) * pi)

In [169]:
password

628